In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append('../src')
from data.audio_dataset import AudioSpectrogramDataset
from models.GAN import GAN

import torch
import librosa
import soundfile as sf
from torch.utils.data import DataLoader

In [6]:
# # hyperparameters
# z_dim = 100
batch_size = 32
# num_epochs = 100
# learning_rate = 0.0002
# beta1 = 0.5

# base_directory = "../data/raw/NSynth/audio"
base_directory = "../data/raw/Bass"

dataset = AudioSpectrogramDataset(base_directory=base_directory, spectro_type='mel')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
data_shape = dataset.get_data_size()
height, width = data_shape[1], data_shape[2]

In [8]:
MusicGAN = GAN(height=height, width=width, latent_dim=100, loss='MSE')

MusicGAN.train(dataloader=dataloader)

[Epoch 0/1] [Batch 0] [D loss: 0.2493998408317566] [G loss: 0.24918651580810547]
[Epoch 0/1] [Batch 1] [D loss: 0.23651286959648132] [G loss: 0.24949684739112854]
[Epoch 0/1] [Batch 2] [D loss: 0.23432797193527222] [G loss: 0.24987109005451202]
[Epoch 0/1] [Batch 3] [D loss: 0.213868647813797] [G loss: 0.2503052353858948]
[Epoch 0/1] [Batch 4] [D loss: 0.20401465892791748] [G loss: 0.2507786750793457]
[Epoch 0/1] [Batch 5] [D loss: 0.19374632835388184] [G loss: 0.25162285566329956]
[Epoch 0/1] [Batch 6] [D loss: 0.199236661195755] [G loss: 0.2526068389415741]
[Epoch 0/1] [Batch 7] [D loss: 0.19072884321212769] [G loss: 0.2543337643146515]
[Epoch 0/1] [Batch 8] [D loss: 0.19822654128074646] [G loss: 0.2557256817817688]
[Epoch 0/1] [Batch 9] [D loss: 0.1692313700914383] [G loss: 0.2575131952762604]
[Epoch 0/1] [Batch 10] [D loss: 0.16361647844314575] [G loss: 0.25981414318084717]
[Epoch 0/1] [Batch 11] [D loss: 0.17606239020824432] [G loss: 0.262114018201828]
[Epoch 0/1] [Batch 12] [D lo

In [ ]:
MusicGAN.generator.eval()

In [7]:
z = torch.randn(1, MusicGAN.latent_dim).to(MusicGAN.device)
generated_spectrogram = MusicGAN.generator(z)

In [8]:
spectrogram = generated_spectrogram.cpu().detach().numpy()
spectrogram = spectrogram.squeeze()

In [9]:
audio = librosa.feature.inverse.mel_to_audio(spectrogram)

path = '../models/generated_samples/'
file_name = 'generated_audio.wav'

sf.write(path + file_name, audio, 22050)